In [3]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
print(os.listdir('.'))

['.ipynb_checkpoints', 'train.json', 'test.json', 'sample_submission.csv', 'README.md', 'Untitled.ipynb', '.git']


In [8]:
training_data = pd.read_json('./train.json')
testing_data = pd.read_json('./test.json')

In [9]:
print(training_data.keys())

Index(['id', 'cuisine', 'ingredients'], dtype='object')


In [14]:
print(training_data['cuisine'])

0              greek
1        southern_us
2           filipino
3             indian
4             indian
            ...     
39769          irish
39770        italian
39771          irish
39772        chinese
39773        mexican
Name: cuisine, Length: 39774, dtype: object


In [15]:
print(training_data['ingredients'])

0        [romaine lettuce, black olives, grape tomatoes...
1        [plain flour, ground pepper, salt, tomatoes, g...
2        [eggs, pepper, salt, mayonaise, cooking oil, g...
3                      [water, vegetable oil, wheat, salt]
4        [black pepper, shallots, cornflour, cayenne pe...
                               ...                        
39769    [light brown sugar, granulated sugar, butter, ...
39770    [KRAFT Zesty Italian Dressing, purple onion, b...
39771    [eggs, citrus fruit, raisins, sourdough starte...
39772    [boneless chicken skinless thigh, minced garli...
39773    [green chile, jalapeno chilies, onions, ground...
Name: ingredients, Length: 39774, dtype: object
